In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

### CNN 훈련 프로세스 개선하기
- CNN 네트워크의 경사 하강 알고리즘의 단점을 해결하는 다양한 방법에 대해 알아보고자 한다.
- CNN 네트워크에서의 과적합을 피할 수 있는 방법에 대해 알아보고자 한다.  


#### 훈련 속도와 Tradeoff
1. 학습률을 높게 설정하면 훈련된 네트워크가 훈련이 반복될 때마다 매개변수가 크게 업데이트 되기 때문에 빠르게 수렴하기는 한다. 그러나 이렇게 되면 네트워크가 적절한 최소 손실값을 찾지를 못하게 된다.
    - 즉, 학습률이 과도하게 낮으면 수렴하는데에 시간이 너무 오래 걸리고
    - 학습률이 과도하게 높으면 수렴을 너무 빨리 하여 극솟값을 지나쳐 버릴 수가 있다.
2. 따라서 이에 대한 해결책으로 학습중에 학습률을 동적으로 조정하는 것이 가능하다. 다음 단계로 넘어갈 때마다 학습률을 감소시켜서 (최솟값에 도달할 수록 더 주의를 기울여 업데이트 하기 위해) learning rate decay의 과정을 거친다.

#### 준최적 극솟값 (Suboptimal Local Minima)
1. 이는 분명히 더 나은 손실함수의 극솟값이 존재하는데에도 불구하고 sgd의 오류에 의해 차선책인 극솟값을 벗어나지 못하는 상황을 의미한다.
    - 이런 상황은 SGD가 랜덤하게 훈련 샘플을 샘플링하기 때문에 즉, 경사가 종종 미니 배치와 다르기 때문에 얕은 극솟값에서 벗어나는 것이 충분히 가능하다. 
    - 물론 항상 최솟값으로 수렴할 것이라는 보장은 아무리 SGD regression을 사용하더라도 불가능하다.  
    
#### 고급 최적화 기법   

1. 모멘텀 알고리즘 (Momentum Algorithm)
    - 이는 SGD를 기반으로 작용한다.
    - 즉, 이전에 업데이트한 매개변수를 받아 새롭게 업데이트를 할 때에 반영하게 되는 것이다.
    - 원래는 그냥 매개변수에 학습률과 gradient값을 곱하여서 빼주었다면 0에서 1사이의 값인 momentum 값을 이전의 업데이트된 항에 곱해서 더해주게 되는 것이다
        - 그래서 만약에 이전과 현재의 단계가 같은 (벡터가 같은 방향) 방향이라면 momentum값을 더해서 SGD를 그 방향으로 가속화 시킨다.
        - 만약에 방향이 다르다면 momentum은 반대로 진동을 약화시키게 될 것이다.
    - 직접 훈련 단계를 구현한 수도 있고 keras의 momentum 모듈을 이용할 수도 있다.
   

In [5]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01, momentum = 0.9, decay = 0.0, nesterov = False)

In [6]:
@tf.function
def train_steps(batch_images, batch_labels):
    with tf.GradientTape() as tape:
        batch_predict = model(batch_images, training = True)
        batch_loss = tf.losses.MSE(batch_labels - batch_predict)
    gradient = tape.gradient(batch_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))
    
    

tf.keras.optimizers.SGD를 보면```nesterov```라는 매개 변수가 존재한다. 이는 True로 설정해 주면 일반적인 momentum algorithm을 Nesterov's algorithm으로 바꾸어 준다.   

2. Nesterov's Algorithm

- 이렇게 바꾸어주는 것이 좋은 이유는 모멘텀 메서드의 네트워크가 손실 최솟값에 실제로 가까이 다가갈 때에 누적 momentum값이 너무 많아서 target minimum 값을 놓치는 상황이 발생할 수 있기 때문이다.
- Nesterov's Algorithm을 적용하게 되면 손실이 이전 단계에 따라 업데이트된 매개변수의 값에 의해 유도되는 경사 변화를 적용해서 경사 하강의 속도를 조금 더 높일 수 있다.  



#### Ada군  
- 각 뉴런의 민감도 및 활성화 빈도에 따라 학습률을 조정하는 아이디어에 변화와 반복을 적용한 알고리즘이다.  

1. Adagrad 
- 일반적으로 발견할 수 있는 특징과 연결된 매개변수에 대해서는 자동으로 학습률을 더 빠르게 감소시키고, 드물게 발견하는 특징에 대해서는 느리게 감소시킨다.
- 즉, 매개변수의 업데이트가 잦을수록 업데이트의 크기가 작아지게 되는 것이다.
- 이 optimizer을 사용하면 직접 학습률을 감소시킬 필요가 없다.  

2. Adadelta
- Adagrad에서 반복할 때마다 학습률을 계속 감소시켜서 결국 0이 되게 하는 문제가 발생하여 학습이 안되는 경우가 발생할 수 있다.
- 이를 막기 위해서 매개변수마다 학습률을 나누어 사용되는 요인을 지속적으로 확인하며 학습을 한다.  
    - 이와 비슷한 optimizer로 RMSprop도 존재한다. (단, 대체로 순환 신경망인 RNN에서 사용을 많이 한다)

3. Adam (Adaptive Moment Estimation)
- 매개변수마다 학습률을 조정하기 위해 이전 업데이트 항을 저장할 뿐만 아니라 과거의 momentum값도 기록을 하게 된다.
    - 이와 비슷한 optimizer로 Nadam도 존재한다.

### 정규화 기법
- 과적합을 피하기 위해서는 최적화 단계를 정교화하는 Regularization(정규화)의 과정이 필수이다.  


1. 조기 중단
   - 동일한 작은 훈련 샘플 집합에 대해 너무 여러번 반복하게 되면 당연히 과대 적합이 발생할 수 밖에 없다.
       - 따라서 훈련을 반복할 때마다 검증 손실돠 설정해준 metric을 검증할 때에 ```tf.keras.callbacks.EarlyStop```을 이용해주면 된다  
       
2. L1, L2 정규화
    - 손실에 정규화 강도를 제한하는 인수와 정규화 항을 곱한 값을 더한다.
    - 이렇게 하여 네트워크는 작업 목적을 최적화할 뿐아니라 매개변수가 취할 수 있는 값을 제한하면서 네트워크를 최적화하게 된다.    
    
    
     1. L2 정규화  
    
        - 네트워크가 그 매개변수 값의 제곱의 합을 최소화하도록 한다.
        - 모든 매개변수의 값이 소멸되지만, 제곱항으로 인해서 제일 큰 매개변수는 더 강한 제재를 가하게 된다.
        - 즉, 네트워크가 매개변수의 값을 낮게 유지해서 더 균일하게 분산되게 한다.  
        
        
     2. L2 정규화  
    
        - 네트워크가 그 매개변수 값의 절댓값의 합을 최소화 하도록 한다.
        - 큰 값을 갖는 가중치가 제곱으로 인해 패널티를 부여받는 것은 아니지만 대신 네트워크가 덜 중요한 특징에 연결된 매개변수의 값을 0으로 축소한다.
        - 따라서 덜 중요한 특성을 무시하여 과적합을 방지한다.
        - 이는 네트워크의 용량이 최소화 되어야 하는 경우에 유리할 것이다.
        
        
        
**L1 정규화와 L2 정규화는 전혀 다른 작용을 한다. L1정규화는 덜 중요한 계층에 연결된 매개변수의 값을 0으로 바꾸어 무시하도록, L2는 반대로 계층의 분산을 고르게 유지하도록 하여서 더욱더 영향을 주는 매개변수가 없도록 한다.**
        

In [43]:
from functools import partial


def l2_reg(coef = 1e-2):
    return lambda x: tf.reduce_sum(x**2) * coef


@tf.function 
def conv_layer(x, kernels, bias, s):
    z = tf.nn.conv2d(x, kernels, strides = [1,s,s,1], padding = 'VALID')
    return tf.nn.relu(z + bias)


class SimpleConvolutionLayer(tf.keras.layers.Layer):
    def __init__(self, num_kernels = 32, kernel_size = (3,3), stride = 1, add_bias = True):
        super().__init__()
        self.num_kernels = num_kernels
        self.kernel_size = kernel_size
        self.stride = stride
    
    def build(self, input_shape):
        self.channel_num = input_shape[-1]
        kernel_shape = (*self.kernel_size, self.channel_num, self.num_kernels)
        glorot_init = tf.keras.initializers.GlorotUniform()
        self.kernels = self.add_weight(
            shape = kernel_shape,
            initializer = glorot_init,
            trainable = True
        )
        self.bias = self.add_weight(
            shape = (self.num_kernels,),
            initializer = 'random_normal',
            trainable = True
        )
        
    def call(self, inputs):
        return conv_layer(inputs, self.kernels, self.bias, self.stride)
    
    
        

In [44]:
class ConvwithReg(SimpleConvolutionLayer):
    def __init__(self, num_kernels = 32, kernel_size = (3,3), strides = 1, kernel_regularization = l2_reg(), bias_regularizer = None):
        super().__init__(num_kernels, kernel_size, strides)
        self.kernel_regularizer = kernel_regularization
        self.bias_regularizer = bias_regularizer
    
    def build(self, input_shape):
        super().build(input_shape)
        if self.kernel_regularizer:
            self.add_loss(partial(self.kernel_regularizer, self.kernels))
        if self.bias_regularizer:
            self.add_loss(partial(self.bias_regularizer, self.bias))
    

In [45]:
conv = ConvwithReg(num_kernels = 32, kernel_size = (3,3), strides= 1, kernel_regularization = l2_reg(1.), bias_regularizer = l2_reg(1.))
conv.build(input_shape= tf.TensorShape((None, 28,28,1)))

**아래의 단계에서 계속 막혔었는데 그 이유는 위에서 SimpleConvolutionLayer의 kernel_shape를 구성할 때에 kernel의 개수, 즉 filter의 개수가 마지막에 가야하는데 차원수와 자리가 바뀌었기 때문이다.**

In [46]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape = (28,28,1)))
model.add(ConvwithReg())
model.add(ConvwithReg())
model.add(ConvwithReg())

In [50]:
class LeNet5(tf.keras.models.Model):
    def __init__(self, num_classes, kernel_regularizer = l2_reg(), bias_regularizer = l2_reg()):
        super(LeNet5, self).__init__()
        self.conv1 = ConvwithReg(6, kernel_size = (5,5))
        self.conv2 = ConvwithReg(16, kernel_size = (5,5))
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size = (2,2))
        self.flatten = tf.keras.layers.Flatten()
        self.Dense1 = tf.keras.layers.Dense(64, activation = 'relu')
        self.Dense2 = tf.keras.layers.Dense(32, activation = 'relu')
        self.Dense3 = tf.keras.layers.Dense(num_classes, activation = 'softmax')
    
    def call(self, x):
        x = self.max_pool(self.conv1(x))
        x = self.max_pool(self.conv2(x))
        x = self.flatten(x)
        x = self.Dense3(self.Dense2(self.Dense1(x)))
        return x

In [51]:
model = LeNet5(10, l2_reg(1.), l2_reg(1.))

In [53]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0
input_shape = (28,28,1)
x_train = x_train.reshape(x_train.shape[0], *input_shape)
x_test = x_test.reshape(x_test.shape[0], *input_shape)

- 그냥 model.fit(x_train, y_train)으로 진행을 하여도 상관이 없기는 하지만 직접 training 과정을 구현해 보는 것이 과정을 제대로 파악하는데에 도움이 될 수 있을 것이다.
- 그런데 일단 중요한 것은 지금 사용하고 있는 reDNN중에서 tf.nn.conv2d가 작동이 안되기 때문에 성능 파악이 어렵다는 것이다.

In [3]:
BATCH_SIZE = 32
num_classes=  10
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000)
dataset = dataset.repeat().batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

In [20]:
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.layers.Conv2D(input_shape = (28,28,1),filters = 32, kernel_size = (5,5), strides = 1, padding = 'valid', activation = 'relu'))
model2.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
model2.add(tf.keras.layers.Conv2D(kernel_size = (5,5),filters = 16, strides = 1, padding = 'valid', activation = 'relu'))
model2.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(256, activation = 'relu'))
model2.add(tf.keras.layers.Dense(128, activation = 'relu'))
model2.add(tf.keras.layers.Dense(num_classes, activation = 'softmax'))

In [21]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 16)          12816     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

In [8]:
class EarlyStopping():
    def __init__(self, patience=0, verbose=0):
        self._step = 0
        self._loss = float('inf')
        self.patience  = patience
        self.verbose = verbose

    def validate(self, loss):
        if self._loss < loss:
            self._step += 1
            if self._step > self.patience:
                if self.verbose:
                    print(f'Training process is stopped early....')
                return True
        else:
            self._step = 0
            self._loss = loss

        return False

In [23]:
opt = tf.keras.optimizers.SGD(momentum = 0.9, learning_rate = 0.01, nesterov = True)
earl_stop = tf.keras.callbacks.EarlyStopping(patience= 3, verbose = 1,monitor= 'accuracy')
model2.compile(loss = 'sparse_categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])


In [24]:
model2.fit(dataset, steps_per_epoch = len(x_train)//BATCH_SIZE, validation_data = val_dataset,epochs = 10, callbacks = [earl_stop])

Train for 1875 steps, validate for 313 steps
Epoch 1/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1706 - accuracy: 0.9472 - val_loss: 0.0601 - val_accuracy: 0.9796
Epoch 2/10
1875/1875 [==============================] - 29s 16ms/step - loss: 0.0517 - accuracy: 0.9837 - val_loss: 0.0424 - val_accuracy: 0.9864
Epoch 3/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0349 - accuracy: 0.9888 - val_loss: 0.0396 - val_accuracy: 0.9876
Epoch 4/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0259 - accuracy: 0.9919 - val_loss: 0.0390 - val_accuracy: 0.9870
Epoch 5/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0192 - accuracy: 0.9939 - val_loss: 0.0324 - val_accuracy: 0.9896
Epoch 6/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0165 - accuracy: 0.9948 - val_loss: 0.0306 - val_accuracy: 0.9906
Epoch 7/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.

**아래와 같이 손실 정규화를 반영하여 과적합을 줄일 수 있도록 하는 훈련 함수를 짜는 것이 가능하다.**

In [26]:
def train_steps():
    for epoch in range(epochs):
        for (batch_img, batch_labels) in dataset:
            with tf.GradientTape() as tape:
                predict = model(batch_img)
                loss = tf.losses.sparse_categorical_crossentropy(batch_labels, predict)
                #L1, L2정규화와 같이 손실의 값을 계속 다음 계층에도 더해주어서 정규화를 시도해 준다.
                #이렇게 할 수 있는 이유는 keras를 이용해서 model.losses속성을 호출하여 손실값 리스트가 반환이 가능하기 떄문이다.
                loss += sum(model.losses)
            gradient = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(loss, model.trainable_variables))
            

그러나 사실은 함수를 굳이 따로 만들거나 하지 않아도  
```l1_reg = tf.keras.regularizers.l1(0.01)``` 을 이용해서 regulatization을 정의하고 (0.01은 정규화의 강도를 제어하는 인수이다.)
```model.add(tf.keras.layers.Conv2D(filters = 6, kernel_size = (3,3), padding = 'same', activation = 'relu', 
input_shape= input_shape, kernel_regularizer = l1_reg))```  
이런 식으로 각 계층마다 kernel의 각각의 값에 대해서 regularization이 가능하게 된다.

#### Dropout
- 인위적으로 0.1에서 0.5 사이의 비율로 네트워크의 뉴런을 훼손시킨다.
- 이는 예측을 위해 데이터의 중복 표현을 개발하는 효과가 있다.

#### Batch Normalization
- 이도 dropout처럼 신경망에 삽입되어서 훈련에 영향을 줄 수 있는 연산이다.
- 이는 이전 계층의 배치 결과를 취해 정규화, 즉 **배치 평균을 빼서 확률값이 배치 표준편차로 나누어 표준 정규분포의 형태를 따르도록 한다.**
- 이 단계는 부수적으로 경사가 네트워크를 통해서 이동하는 방법을 개선하고 SGD 프로세스를 가능하게 한다.